# Twin4Build: CO2 Controller Example

This notebook demonstrates how to use the Twin4Build package to create and simulate a CO2 controller model for a building space. We'll go through the process step-by-step, explaining each part along the way.

## 1. Import Required Libraries

First, we import the necessary libraries and modules.

In [1]:
%pip install git+https://github.com/JBjoernskov/Twin4Build.git # Uncomment in google colab
import datetime
import sys
sys.path.append(r"C:\Users\jabj\Documents\python\Twin4Build")
from dateutil import tz
import twin4build as tb


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


## 2. Define Model Components and Connections

Now we define the components of our CO2 controller model and their connections.<br>
For convenience, we can pack all these definitions into the "fcn" function, which will be used to create the model.

In [2]:
def fcn(self):
    ##############################################################
    ################## First, define components ##################
    ##############################################################
    occupancy_schedule = tb.ScheduleSystem(
        weekDayRulesetDict={
            "ruleset_default_value": 0,
            "ruleset_start_minute": [0, 0, 0, 0, 0, 0, 0],
            "ruleset_end_minute": [0, 0, 0, 0, 0, 0, 0],
            "ruleset_start_hour": [6, 7, 8, 12, 14, 16, 18],
            "ruleset_end_hour": [7, 8, 12, 14, 16, 18, 22],
            "ruleset_value": [3, 5, 20, 25, 27, 7, 3]},
        add_noise=True,
        saveSimulationResult=True,
        id="Occupancy schedule")
    
    co2_setpoint_schedule = tb.ScheduleSystem(
        weekDayRulesetDict={
            "ruleset_default_value": 900,
            "ruleset_start_minute": [],
            "ruleset_end_minute": [],
            "ruleset_start_hour": [],
            "ruleset_end_hour": [],
            "ruleset_value": []},
        saveSimulationResult=True,
        id="CO2 setpoint schedule")

    co2_controller = tb.PIControllerFMUSystem(
        kp=0.001,
        Ti=3,
        isReverse=False,
        saveSimulationResult=True,
        id="CO2 controller")

    supply_damper = tb.DamperSystem(
        nominalAirFlowRate=1.6,
        a=5,
        saveSimulationResult=True,
        id="Supply damper")

    return_damper = tb.DamperSystem(
        nominalAirFlowRate=1.6,
        a=5,
        saveSimulationResult=True,
        id="Return damper")

    space = tb.BuildingSpaceCo2System(
        airVolume=466.54,
        outdoorCo2Concentration=500,
        infiltration=0.005,
        generationCo2Concentration=0.0042*1000*1.225,
        saveSimulationResult=True,
        id="Space")

    #################################################################
    ################## Add connections to the model #################
    #################################################################
    self.add_connection(co2_controller, supply_damper,
                         "inputSignal", "damperPosition")
    self.add_connection(co2_controller, return_damper,
                         "inputSignal", "damperPosition")
    self.add_connection(supply_damper, space,
                         "airFlowRate", "supplyAirFlowRate")
    self.add_connection(return_damper, space,
                         "airFlowRate", "returnAirFlowRate")
    self.add_connection(occupancy_schedule, space,
                         "scheduleValue", "numberOfPeople")
    self.add_connection(space, co2_controller,
                         "indoorCo2Concentration", "actualValue")
    self.add_connection(co2_setpoint_schedule, co2_controller,
                         "scheduleValue", "setpointValue")

## 3. Create and Load the Model

Now we create a model instance and load it with the components and connections we defined.<br>
You can compare the shown graph and model structure with the one we defined in "fcn" function as part of the previous step.

In [3]:
model = tb.Model(id="co2_control")
model.load(fcn=fcn)
# system_graph = os.path.join(model.graph_path, "system_graph.png")
# image = plt.imread(system_graph)
# plt.figure(figsize=(12,12))
# plt.imshow(image)
# plt.axis('off')
# plt.show()

C:\Users\jabj\Documents\python\Twin4Build\twin4build\utils\mkdir_in_root.py:5: MainPathNotFound: Could not determine main module path, using temp dir: C:\Users\jabj\AppData\Local\Temp\1
  current_dir = get_main_dir()


Model related files are stored in the following directory:
 C:\Users\jabj\AppData\Local\Temp\1\generated_files\models\co2_control

Loading model...[OK]
   |___Drawing simulation model...
Loading model...[OK]
   |___Applying user defined function...

AttributeError: module 'twin4build' has no attribute 'BuildingSpaceCo2System'

## 4. Set Up Simulation Parameters and Run Simulation

We set up the simulation parameters and run the simulation for our CO2 controller model.

In [ ]:
simulator = tb.Simulator()
stepSize = 600 #Seconds
startTime = datetime.datetime(year=2024, month=1, day=10, hour=0, minute=0, second=0, tzinfo=tz.gettz("Europe/Copenhagen"))
endTime = datetime.datetime(year=2024, month=1, day=12, hour=0, minute=0, second=0, tzinfo=tz.gettz("Europe/Copenhagen"))

# Simulate the model
simulator.simulate(model,
                    stepSize=stepSize,
                    startTime = startTime,
                    endTime = endTime)

## 5. Visualize Results

We use the built-in plotting functions to visualize the simulation results.

In [ ]:
plot.plot_component(simulator, 
                    components_1axis=[("Supply damper", "airFlowRate")], 
                    components_2axis=[("Supply damper", "damperPosition")], 
                    ylabel_1axis="Massflow [kg/s]", #Optional
                    ylabel_2axis="Damper position", #Optional
                    show=True, 
                    align_zero=True,
                    nticks=11)

In [ ]:
plot.plot_component(simulator, 
                    components_1axis=[("CO2 controller", "inputSignal")], 
                    components_2axis=[("CO2 controller", "actualValue"), 
                                      ("CO2 controller", "setpointValue")],
                    ylabel_1axis="Position", #Optional
                    ylabel_2axis=r"CO$_2$ concentration [ppm]", #Optional
                    show=True,
                    nticks=11)

In [ ]:
plot.plot_component(simulator, 
                    components_1axis=[("Space", "indoorCo2Concentration")], 
                    components_2axis=[("Space", "numberOfPeople")],
                    components_3axis=[("Space", "supplyAirFlowRate")], 
                    ylabel_1axis=r"CO$_2$ concentration [ppm]", #Optional
                    ylabel_2axis="Number of people", #Optional
                    ylabel_3axis=r"Airflow [kg/s]", #Optional
                    show=True,
                    nticks=11)

## 6. Interpreting the Results

The plots show:

1. Supply Damper Position: This graph illustrates how the damper position and airflow changes over time in response to CO2 levels.

2. CO2 Controller: This plot shows the actual CO2 concentration compared to the setpoint, and the controller output (green line).

These visualizations help us understand how the CO2 controller is managing the indoor air quality based on occupancy and setpoints.

## 7. Next Steps

To further explore this model, you could:
- Adjust the PID controller parameters to see how they affect the system response
- Modify the occupancy schedule to simulate different usage patterns
- Change the CO2 setpoint to observe its impact on ventilation behavior

This example demonstrates the power of Twin4Build for modeling and simulating building control systems.